# Notebook to extract negative labels for the energy patent classifier.
The process chosen is to create 100 clusters and use data from clusters without any postive data.  
There are another methods more precise and reliables (hierarchical cluster from positive data and using bert embedings) that have not been used because of time constraint

In [0]:

names = [
    "US07820139B2.xml", "US20130002962A1.xml", "US20130018608A1.xml", "US20130060981A1.xml", "US20130064229A1.xml",
    "US20130094447A1.xml", "KR101894541B1.xml", "KR20180111181A.xml", "US20150219357A1.xml", "US20150241078A1.xml",
    "US20150277467A1.xml", "US20150319689A1.xml", "US20160007280A1.xml", "US20160070249A1.xml", "DE102008040440A1.xml",
    "DE102008042273A1.xml", "DE102009012488A1.xml", "DE102013202978A1.xml", "DE102017009546A1.xml", "DE19841922A1.xml",
    "DE2942932A1.xml", "DE3136273A1.xml", "FR3023438A1.xml", "FR3023438B1.xml", "FR3060930A1.xml", "IN2014CN07341A.xml",
    "JP2015056104A.xml", "JP2015527862A.xml", "US20010036222A1.xml", "US20030133031A1.xml", "US20030171851A1.xml",
    "US20050038571A1.xml", "US20050043862A1.xml", "US20160334030A1.xml", "US20170108905A1.xml", "US20170189889A1.xml",
    "US20170222437A1.xml", "US09810442B2.xml", "US09854515B2.xml", "US09867654B2.xml", "US09989949B2.xml",
    "US10054918B2.xml", "US20170288401A1.xml", "US20170324908A1.xml", "US20170332750A1.xml", "US20170341942A1.xml",
    "US20170343229A1.xml", "US20180031449A1.xml", "US20180173024A1.xml", "US20180206131A1.xml", "US20180274334A1.xml",
    "US20180289120A1.xml", "US20180291886A1.xml", "US20180294248A1.xml", "US20180310858A1.xml", "US20180326173A1.xml",
    "US10136669B2.xml", "US10139845B2.xml", "US10144586B2.xml"]
print(f"Hay {len(names)} registros")

Hay 59 registros

In [0]:
import pyspark.sql.functions as sf
FILTERED_STORAGE_NAME = "challengebasf"
FILTERED_CONTAINER_NAME = "filtereddata"
FILTERED_OUTPUT_FOLDER = "output_data"
input_container = f"wasbs://{FILTERED_CONTAINER_NAME}@{FILTERED_STORAGE_NAME}.blob.core.windows.net"
input_path = f"{input_container}/{FILTERED_OUTPUT_FOLDER}/"

In [0]:
df = spark.read.parquet(input_path)

In [0]:
# Clean and tokenize -> move to spark nlp
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer
OUTPUT_COL_ENGLISH_TEXT = "english_text"
df_clean = df.select('_file', OUTPUT_COL_ENGLISH_TEXT)
tokenizer = Tokenizer(inputCol=OUTPUT_COL_ENGLISH_TEXT, outputCol="text_token")
df_words_token = tokenizer.transform(df_clean)
remover = StopWordsRemover(inputCol="text_token", outputCol="text_clean")
df_words_no_stopw = remover.transform(df_words_token)

In [0]:
minDF = 5
cv = CountVectorizer(inputCol="text_clean", outputCol="features", minDF=minDF)
cv_model = cv.fit(df_words_no_stopw)
vectorized_tokens = cv_model.transform(df_words_no_stopw)

In [0]:
from pyspark.ml.clustering import LDA

num_topics = 50
lda = LDA(k=num_topics, maxIter=20)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print(f"The lower bound on the log likelihood of the entire corpus: {ll}")
print(f"The upper bound on perplexity: {lp}")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-211828917365332> in <module> 
 4 lda = LDA ( k = num_topics , maxIter = 20 ) 
 5 model = lda . fit ( vectorized_tokens ) 
 ----> 6 ll = model . logLikelihood ( vectorized_tokens ) 
 7 lp = model . logPerplexity ( vectorized_tokens ) 
 8 print ( "The lower bound on the log likelihood of the entire corpus: " + str ( ll ) ) 

 /databricks/spark/python/pyspark/ml/clustering.py in logLikelihood (self, dataset) 
 758 : py : func : ` topicsMatrix ` to the driver . This implementation may be changed in the future . 
 759 """
 --> 760 return self . _call_java ( "logLikelihood" , dataset ) 
 761 
 762 @ since ( "2.0.0" ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _call_java (self, name, *args) 
 53 sc = SparkContext . _active_spark_context
 54 java_args = [ _py2java ( sc , arg ) for arg in args ] 
 ---> 55 return _java2py ( sc , m ( * java_args ) ) 
 56 
 57 @ staticmethod

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o9192.logLikelihood.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 219 in stage 68.0 failed 4 times, most recent failure: Lost task 219.3 in stage 68.0 (TID 11573, 10.139.64.8, executor 8): com.databricks.sql.io.FileReadException: Error while reading file wasbs:REDACTED_LOCAL_PART@challengebasf.blob.core.windows.net/output_data/part-00060-tid-5463204523206640381-35ffc0e8-c68d-4c2f-befd-6615bb13b4dc-1459-1-c000.snappy.parquet.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:331)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:310)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anonfun$prepareNextFile$1.apply(FileScanRDD.scala:463)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anonfun$prepareNextFile$1.apply(FileScanRDD.scala:451)
	at scala.concurrent.impl.Future$PromiseCompletingRunnable.liftedTree1$1(Future.scala:24)
	at scala.concurrent.impl.Future$PromiseCompletingRunnable.run(Future.scala:24)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable$$anonfun$run$1.apply$mcV$sp(SparkThreadLocalForwardingThreadPoolExecutor.scala:100)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable$$anonfun$run$1.apply(SparkThreadLocalForwardingThreadPoolExecutor.scala:100)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable$$anonfun$run$1.apply(SparkThreadLocalForwardingThreadPoolExecutor.scala:100)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper$class.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:97)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:100)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolE

In [0]:
# extract vocabulary from CountVectorizer
vocab = cv_model.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
container
connector
type
assembly
block
structure
joint
electrical
elevator
training
*************************
topic: 1
*************************
cutting
appliance
household
bicycle
method
moving
picture
domestic
modulation
cooking
*************************
topic: 2
*************************
therapy
methods
artificial
systems
web
cancer
nail
method
noninvasive
system
*************************
topic: 3
*************************
game
machine
hybrid
wagering
chair
dryer
gaming
system
transducer
feature
*************************
topic: 4
*************************
vehicle
multifunction
trailer
method
headlight
device
system
pallet
aquaculture
headrest
*************************
topic: 5
*************************
multiple
folding
protective
attachment
device
boot
disease
footwear
method
solutions
*************************
topic: 6
*************************
umbrella
gun
piezoelectric
thickness
compensator
method
rim
engaging
hook
tissue
*************************
topic: 7
*************************
valve
plug
electroluminescent
anchor
concrete
check
spark
organic
method
connecting
*************************
topic: 8
*************************
method
vegetables
photoreceptor
fruits
apparatus
electrophotographic
system
egg
mosquito
cotton
*************************
topic: 9
*************************
printer
refrigerator
cigarette
method
retainer
inkjet
gravity
drawer
atomizer
printhead
*************************
topic: 10
*************************
method
manufacturing
thereof
preparation
device
process
material
producing
using
production
*************************
topic: 11
*************************
disposable
microwave
oven
injector
new
wear
method
absorbent
aquatic
jack
*************************
topic: 12
*************************
light
emitting
package
surgical
chip
laser
diode
device
system
organic
*************************
topic: 13
*************************
nonaqueous
electrolyte
solution
electrolytic
battery
secondary
method
system
fireplace
apparatus
*************************
topic: 14
*************************
wind
generator
turbine
power
manhole
cover
brake
steering
method
shock
*************************
topic: 15
*************************
tester
method
n
liquids
device
ionic
stretcher
lipid
system
apparatus
*************************
topic: 16
*************************
treatment
method
apparatus
cleaning
medical
device
treating
system
processing
substrate
*************************
topic: 17
*************************
golf
head
ball
robot
club
management
method
system
methods
systems
*************************
topic: 18
*************************
method
counter
centrifuge
system
vaporizer
sight
precipitator
normalization
computation
communication
*************************
topic: 19
*************************
compact
synthesis
method
hardware
device
luminous
apparatus
pan
imaging
prop
*************************
topic: 20
*************************
acid
use
thereof
method
protein
human
antibodies
methods
nucleic
gene
*************************
topic: 21
*************************
method
system
data
apparatus
systems
methods
network
based
using
storage
*************************
topic: 22
*************************
drying
arm
bracket
card
shoe
mounting
replaceable
hanger
clothes
device
*************************
topic: 23
*************************
imaging
camera
safety
seat
detector
apparatus
system
tape
inspection
radiation
*************************
topic: 24
*************************

derivatives
d
method
device
inhibitors
substituted
h
inhibitor
shoulder
*************************
topic: 25
*************************
transformer
therapeutic
method
stabilized
agent
methods
microorganisms
tailored
device
oils
*************************
topic: 26
*************************
compounds
novel
use
inhibitors
method
uses
transcription
thereof
ram
turbine
*************************
topic: 27
*************************
paper
coil
drug
device
cylinder
feeding
method
goods
delivery
book
*************************
topic: 28
*******

In [0]:
result = model.transform(vectorized_tokens)


In [0]:
output_container_path = f"wasbs://otros@{FILTERED_STORAGE_NAME}.blob.core.windows.net/lda_model_{minDF}/"
model.save(output_container_path)

In [0]:
output_container_path = f"wasbs://otros@{FILTERED_STORAGE_NAME}.blob.core.windows.net"
output_blob_folder = f"{output_container_path}/lda_results_{minDF}/"
result.write.mode("overwrite").parquet(output_blob_folder)
print(f"Data saved!")

Data saved!

In [0]:
result.printSchema()

root
-- _file: string (nullable = true)
-- english_text: string (nullable = true)
-- text_token: array (nullable = true)
 |-- element: string (containsNull = true)
-- text_clean: array (nullable = true)
 |-- element: string (containsNull = true)
-- features: vector (nullable = true)
-- topicDistribution: vector (nullable = true)

In [0]:
result.show()

+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 _file| english_text| text_token| text_clean| features| topicDistribution|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
CA2801779A1.xml|powder supplying ...|[powder, supplyin...|[powder, supplyin...|(36490,[0,1,148,2...|[0.00198946729277...|
CA3016202A1.xml|rechargeable sodi...|[rechargeable, so...|[rechargeable, so...|(36490,[23,45,57,...|[0.00221073156651...|
CA2831236A1.xml|method and appara...|[method, and, app...|[method, apparatu...|(36490,[0,3,53,98...|[0.00248737169732...|
CA3015961A1.xml|machining swarf r...|[machining, swarf...|[machining, swarf...|(36490,[3,368,630...|[0.00398237733892...|
CA2874902A1.xml|enhanced channel ...|[enhanced, channe...|[enhanced, channe...|(36490,[6,86,132,...|[0.00153005421121...|
CA3013760A1.xml|compositions and ...|[compositions, an...|[compositions, me...|(36490,[4,72,130,...|[0.00398237733892...|
CA2983433A1.xml|manufacturing pro...|[manufacturing, p...|[manufacturing, p...|(36490,[10,24,757...|[0.00398237733892...|
CA3024754A1.xml|apparatus and met...|[apparatus, and, ...|[apparatus, metho...|(36490,[0,3,309,4...|[0.00331769091057...|
CA2370887A1.xml|production of the...|[production, of, ...|[production, lipi...|(36490,[54,485,15...|[0.00248737169732...|
CA2972418A1.xml|mass cytometry ap...|[mass, cytometry,...|[mass, cytometry,...|(36490,[3,4,804,8...|[0.00398237733892...|
 CA2660817C.xml|solar roof tile w...|[solar, roof, til...|[solar, roof, til...|(36490,[50,54,66,...|[0.00180846450141...|
CA3015439A1.xml|water management ...|[water, managemen...|[water, managemen...|(36490,[2,36,46,4...|[0.00180846450141...|
CA3025438A1.xml|endomorphin tetra...|[endomorphin, tet...|[endomorphin, tet...|(36490,[8,251,276...|[0.00331769091057...|
CA3023595A1.xml|method and device...|[method, and, dev...|[method, device, ...|(36490,[0,1,136,7...|[0.00221073156651...|
 CA2737039C.xml|apparatus and met...|[apparatus, and, ...|[apparatus, metho...|(36490,[0,3,87,14...|[0.00180846450141...|
 CA2625626C.xml|material composit...|[material, compos...|[material, compos...|(36490,[0,5,34,72...|[0.00165765057164...|
CA2918870A1.xml|grip elements for...|[grip, elements, ...|[grip, elements, ...|(36490,[331,385,2...|[0.52725038820092...|
CA2960087A1.xml|methods and compo...|[methods, and, co...|[methods, composi...|(36490,[4,72,379,...|[0.00221073156651...|
 CA2701915C.xml|fluid scrubber an...|[fluid, scrubber,...|[fluid, scrubber,...|(36490,[61,151,62...|[0.00248737169732...|
 CA2635628C.xml|annular seal and ...|[annular, seal, a...|[annular, seal, p...|(36490,[61,126,51...|[0.00398237733892...|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

# Comprobación de resultados

We are going to check:
- If data has been scattered between all clusters
- Distribution of positive data between clusters
- Identification of clusters to select data as negative registers

In [0]:
from pyspark.sql.types import *
import numpy as np

In [0]:
minDF = 5
output_container_path = f"wasbs://otros@{FILTERED_STORAGE_NAME}.blob.core.windows.net"
output_blob_folder = f"{output_container_path}/lda_results_{minDF}/"
df_raw = spark.read.parquet(output_blob_folder)
df = df_raw

In [0]:
df = df.withColumn("positive_data", sf.when(sf.col("_file").isin(names), 1).otherwise(0))
to_array = sf.udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))
df = df.withColumn("topicDistribution", to_array("topicDistribution"))

for n in range(0,50):
  df = df.withColumn(f"topic_{n}", sf.col("topicDistribution").getItem(n))
  
@sf.udf(IntegerType())
def get_cluster(values):
  return max(enumerate(values), key=lambda x: x[1])[0]
df = df.withColumn("cluster", get_cluster(sf.col("topicDistribution")))
num_per_cluster = df.groupBy("cluster").agg(
  sf.sum("positive_data").alias("num_positive_data"), 
  sf.count("cluster").alias("num_total_data"))


In [0]:
num_per_cluster_p = num_per_cluster.toPandas()

In [0]:
assert num_per_cluster_p["num_positive_data"].sum() == len(names)
num_per_cluster_p.loc[:, "perc_positives"] = 100 * num_per_cluster_p["num_positive_data"] / num_per_cluster_p["num_positive_data"].sum()
num_per_cluster_p.loc[:, "perc_total"] = 100* num_per_cluster_p["num_total_data"] / num_per_cluster_p["num_total_data"].sum()

In [0]:
num_per_cluster_p.sort_values("num_positive_data", ascending=False)

,cluster,num_positive_data,num_total_data,perc_positives,perc_total
40,33,36,221885,61.016949,16.647510
5,44,8,145266,13.559322,10.898966
32,10,4,194177,6.779661,14.568644
43,42,3,37514,5.084746,2.814587
38,21,3,179800,5.084746,13.489971
11,16,2,32570,3.389831,2.443650
49,36,1,20956,1.694915,1.572279
47,0,1,40180,1.694915,3.014611
44,2,1,5201,1.694915,0.390219
35,25,0,1237,0.000000,0.092809


In [0]:
num_per_cluster_p.loc[num_per_cluster_p["num_positive_data"] == 0, "perc_total"].sum()

Out[37]: 34.159562798891386

All cluster have registers.  We can see that the LDA has not created a cluster with energy consumption patents (it would have been ideal) but they are scattered between the most populated clusters.  
It is interesting that cluster 33 has 61% of all positive data with only 16% of registers.
Clusters with no positive data have the 34% of all data

In [0]:
positives = df.filter(sf.col("positive_data") == 1)
positives = positives.drop("topicDistribution")
positives = positives.drop("english_text")
positives = positives.drop("text_token")
positives = positives.drop("text_clean")
positives = positives.drop("features")
positives_p = positives.toPandas()

In [0]:
positives_p["max"] = positives_p.drop("cluster", axis=1).max(axis=1)

In [0]:
positives_p.head()

,_file,positive_data,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,topic_30,topic_31,topic_32,topic_33,topic_34,topic_35,topic_36,topic_37,topic_38,topic_39,topic_40,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,cluster,max
0,US20050038571A1.xml,1,0.001808,0.001781,0.001780,0.001783,0.001774,0.001779,0.001773,0.001781,0.001772,0.001774,0.002008,0.001778,0.001799,0.001773,0.001786,0.001772,0.001818,0.001783,0.001772,0.001773,0.001797,0.001928,0.001777,0.001801,0.001782,0.001774,0.001778,0.001781,0.001854,0.001774,0.001796,0.001772,0.001894,0.393589,0.001776,0.001774,0.001801,0.001778,0.001828,0.001773,0.001772,0.001773,0.001814,0.001781,0.520232,0.001814,0.001778,0.001791,0.001777,0.001773,44,1.0
1,DE102009012488A1.xml,1,0.001989,0.001959,0.001958,0.001961,0.001951,0.001957,0.001951,0.001959,0.001950,0.001952,0.222771,0.001955,0.001979,0.001950,0.001964,0.001950,0.002000,0.001962,0.001949,0.001951,0.001977,0.275846,0.001955,0.001982,0.001960,0.001951,0.001956,0.001959,0.002039,0.001952,0.001976,0.001949,0.002083,0.408732,0.001954,0.001952,0.001982,0.001956,0.002011,0.001950,0.001949,0.001951,0.001996,0.001960,0.002176,0.001995,0.001956,0.001970,0.001954,0.001950,33,1.0
2,DE102013202978A1.xml,1,0.000795,0.000783,0.000783,0.000784,0.000780,0.000782,0.000780,0.000783,0.000780,0.000780,0.000883,0.000782,0.000791,0.000780,0.000785,0.000779,0.000800,0.000784,0.000779,0.000780,0.000790,0.000848,0.000782,0.000792,0.000784,0.000780,0.000782,0.000783,0.000815,0.000780,0.000790,0.000779,0.000833,0.000896,0.000781,0.000780,0.000792,0.000782,0.000804,0.000780,0.000779,0.000780,0.000798,0.000783,0.961202,0.000798,0.000782,0.000788,0.000781,0.000780,44,1.0
3,US20130002962A1.xml,1,0.001808,0.001781,0.001780,0.001783,0.001774,0.001779,0.001773,0.001781,0.001772,0.001774,0.002008,0.001778,0.001799,0.001773,0.001786,0.001772,0.001818,0.001783,0.001772,0.001773,0.001797,0.001928,0.001777,0.001801,0.001782,0.001774,0.001778,0.001781,0.001854,0.001774,0.001796,0.001772,0.001894,0.911842,0.001776,0.001774,0.001801,0.001778,0.001828,0.001773,0.001772,0.001773,0.001814,0.001781,0.001978,0.001814,0.001778,0.001791,0.001777,0.001773,33,1.0
4,US10139845B2.xml,1,0.001530,0.001507,0.001506,0.001508,0.001501,0.001505,0.001500,0.001507,0.001500,0.001501,0.001699,0.001504,0.001522,0.001500,0.001511,0.001499,0.001538,0.001509,0.001499,0.001500,0.001521,0.001631,0.001504,0.001524,0.001507,0.001501,0.001504,0.001507,0.001568,0.001501,0.001519,0.001499,0.001602,0.925414,0.001503,0.001501,0.001524,0.001504,0.001547,0.001500,0.001499,0.001500,0.001535,0.001507,0.001673,0.001535,0.001504,0.001515,0.001503,0.001500,33,1.0


In [0]:
positives_p.drop(["_file", "positive_data"],axis=1).sum().sort_values(ascending=False)

Out[49]: cluster 1839.000000
max 59.000000
topic_33 30.643279
topic_44 5.389727
topic_21 3.909683
topic_10 3.363302
topic_42 2.470246
topic_16 1.520468
topic_0 1.129603
topic_46 0.976337
topic_32 0.957912
topic_36 0.719479
topic_2 0.610891
topic_45 0.536993
topic_6 0.505626
topic_14 0.443671
topic_38 0.432891
topic_19 0.334652
topic_47 0.301607
topic_20 0.291726
topic_30 0.279930
topic_23 0.214421
topic_24 0.202587
topic_48 0.171865
topic_28 0.133663
topic_12 0.129732
topic_17 0.128582
topic_3 0.128559
topic_43 0.128435
topic_7 0.128424
topic_1 0.128403
topic_27 0.128392
topic_5 0.128285
topic_26 0.128217
topic_37 0.128195
topic_11 0.128167
topic_22 0.128164
topic_34 0.128078
topic_9 0.127944
topic_35 0.127942
topic_29 0.127939
topic_25 0.127908
topic_4 0.127892
topic_41 0.127869
topic_49 0.127828
topic_39 0.127816
topic_13 0.127806
topic_8 0.127801
topic_15 0.127791
topic_18 0.127778
topic_40 0.127755
topic_31 0.127743
dtype: float64

The clusters with positive info are: 33, 44, 10, 42, 21, 16, 36, 0, 2. we are going to use the last 30 to sample negative data

In [0]:
positives_p.drop(["_file", "positive_data"],axis=1).sum().sort_values(ascending=False)[-30:]

Out[51]: topic_24 0.202587
topic_48 0.171865
topic_28 0.133663
topic_12 0.129732
topic_17 0.128582
topic_3 0.128559
topic_43 0.128435
topic_7 0.128424
topic_1 0.128403
topic_27 0.128392
topic_5 0.128285
topic_26 0.128217
topic_37 0.128195
topic_11 0.128167
topic_22 0.128164
topic_34 0.128078
topic_9 0.127944
topic_35 0.127942
topic_29 0.127939
topic_25 0.127908
topic_4 0.127892
topic_41 0.127869
topic_49 0.127828
topic_39 0.127816
topic_13 0.127806
topic_8 0.127801
topic_15 0.127791
topic_18 0.127778
topic_40 0.127755
topic_31 0.127743
dtype: float64

In [0]:
negative_topics_cluster = positives_p.drop(["_file", "positive_data"],axis=1).sum().sort_values(ascending=False)[-30:].index.tolist()
negative_clusters = [int(topic.split("topic_")[1]) for topic in negative_topics_cluster]
negative_cluster_data = df.filter(sf.col("cluster").isin(negative_clusters))
num_total_negative_registers = negative_cluster_data.count()

In [0]:
target_num_samples = 600
negatives = negative_cluster_data.sample(False, target_num_samples/num_total_negative_registers, seed=18)

In [0]:
assert negatives.columns == df.columns
total = negatives.union(df.filter(sf.col("positive_data") == 1))
total = total.select("_file", "positive_data")

Check the output container exists

In [0]:
output_container_path = f"wasbs://trainingsamples@{FILTERED_STORAGE_NAME}.blob.core.windows.net"
output_blob_folder = f"{output_container_path}/file_names/"
total.write.mode("overwrite").parquet(output_blob_folder)
print(f"Data saved!")

Data saved!